In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using the Vertex AI Gemini API to explain BigQuery ML Clustering


| | |
|----------|-------------|
| Author(s)   | Hussain Chinoy (ghchinoy@) |



## Overview

This example demostrates how to use the Vertex AI Gemini API to explain BigQuery ML clustering.

[BigQuery ML (BQML)](https://cloud.google.com/bigquery/docs/bqml-introduction) is a powerful way for analysts to utilize various ML techniques within a familiar relational data store - BigQuery.

Typically, the end goal is to explain the results of analysis - this is where the Vertex AI Gemini LLM API can come in handy.

### About Clustering
Custering, also called segmentation, is a common exploratory technique to discover different groupings of behavior types based on a collection of known attributes within data, in this case, consumer purchasing patterns. We can imagine people following different purchasing patterns based upon our lived experiences, but using data and BQML in particular provides and uncovers segmentation in interesting and reproducible ways.

Segmentation is used to identify naturally occurring groupings of customers who have similar characteristics, such as purchasing habits based upon stores visited, items purchased, or price paid for those items. Or, in this example, to design a more meaningful marketing campaign that appeals to a specific group.

The aim of customer segmentation is to split customers by attributes and one common method of segmentation is the “RFM model”
* Recency  - How recently did a customer purchase or what’s the avg gap between purchases?
* Frequency - How often did they purchase?
* Monetary value - How much did they spend?

Some other variations are
* RFE - Recency, Frequency, Engagement - visit duration, pages per visit, etc.
* RFD - Recency, Frequency, Duration - length of time spent browsing on a page (advertising use cases)

Using a public data source via Analytics Hub, we'll the publicly available synthetic eCommerce data set to our BigQuery instance, create a cluster, and then explain it with Gemini.


Learn more about [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) and the [Vertex AI Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)


### Objective

In this tutorial, you learn how to use BigQuery ML (BQML) to cluster ecommerce users based on spend history and then explain those clusters using the Gemini Foundational Large Multimodal Model, making cluster analysis more accessible to your team!

This tutorial uses the following Google Cloud ML services and resources:

- BigQuery
- BigQuery ML
- Vertex AI Gemini


The steps performed include:

- Take a peek at the data
- Create a K-means cluster with BQML
- Explain that cluster results with Gemini


### Dataset

For this example, we'll use a [BigQuery Public Dataset](https://console.cloud.google.com/marketplace/browse?filter=partner:BigQuery%20Public%20Data) that contains synthetic eCommerce and Digital Marketing data for the ficitious company called TheLook. Here's the link to the dataset [theLook eCommerce](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce)

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
[Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


Let's log in with Google, load the Vertex AI libraries and restart the runtime

## Setup


### Required SDKs

Let's install the required SDKs

In [1]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install bigframes==0.25 --quiet

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Before proceeding, please wait for the kernel to finish restarting ⚠️</b>
</div>

### Authenticate your Google Account

Depending on your environment, you may have to manually authenticate. Please follow instructions below.

For Vertex AI Workbench, you're already authenticated!

#### Colab

In [4]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()


Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

In [3]:
import sys
import pandas as pd
from typing import Union
from google.cloud import bigquery

In [5]:
#@title Setup Project Variables { run: "auto", display-mode: "form" }
project_id = 'perspective-von-clownstick' #@param {type:"string"}
dataset_name = "bqml_tutorial_us" #@param {type:"string"}
model_name = "ecommerce_customer_segment_cluster5" #@param {type:"string"}
eval_name = model_name + "_eval"
LOCATION = "us-central1"  # @param {type:"string"}
client = bigquery.Client(project=project_id)

# creates the target dataset
try:
    dataset = bigquery.Dataset(f"{project_id}.{dataset_name}")
    dataset = client.create_dataset(dataset)
except:
    print(f"Dataset {dataset_name} already exists. Please continue.")



Dataset bqml_tutorial_us already exists. Please continue.


## Create a K-means model to cluster ecommerce data

First let's look at our data quickly before we create the model. This query can be run in BigQuery on its own. Try it out!

In [6]:
query = """
SELECT
  user_id,
  order_id,
  sale_price,
  created_at as order_created_date
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
"""
df = client.query(query).to_dataframe()
df.head()


,user_id,order_id,sale_price,order_created_date
0,28570,35732,0.02,2022-11-15 13:50:48+00:00
1,96719,120611,0.02,2020-10-29 01:13:19+00:00
2,49285,61888,0.02,2022-02-01 03:19:27+00:00
3,24831,31097,0.02,2022-10-29 15:34:16+00:00
4,56841,71284,0.02,2020-06-23 10:07:08+00:00


## `CREATE MODEL` using `KMEANS`

Create a query then start the model creation job, using a python loop to wait for the job to complete. Please note, if you've created the model already, there's no need to rerun this step to create the clustering using model on the data.

In [7]:
query = """
CREATE OR REPLACE MODEL `{0}.{1}`
OPTIONS (
  MODEL_TYPE = "KMEANS",
  NUM_CLUSTERS = 5,
  KMEANS_INIT_METHOD = "KMEANS++",
  STANDARDIZE_FEATURES = TRUE )
AS (
SELECT * EXCEPT (user_id)
FROM (
  SELECT user_id,
    DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) as DATE), day) AS days_since_order, -- RECENCY
    COUNT(DISTINCT order_id) AS count_orders, -- FREQUENCY
    AVG(sale_price) AS avg_spend -- MONETARY
  FROM (
    SELECT user_id,
      order_id,
      sale_price,
      created_at as order_created_date
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
  )
  GROUP BY user_id, order_id
 )
)
""".format(dataset_name, model_name)


In [8]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [ ]:
print(query)

# this should take under 5 minutes to create the model
run_bq_query(query)


CREATE OR REPLACE MODEL `bqml_tutorial_us.ecommerce_customer_segment_cluster5`
OPTIONS (
  MODEL_TYPE = "KMEANS",
  NUM_CLUSTERS = 5,
  KMEANS_INIT_METHOD = "KMEANS++",
  STANDARDIZE_FEATURES = TRUE )
AS (
SELECT * EXCEPT (user_id)
FROM (
  SELECT user_id,
    DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) as DATE), day) AS days_since_order, -- RECENCY
    COUNT(DISTINCT order_id) AS count_orders, -- FREQUENCY
    AVG(sale_price) AS avg_spend -- MONETARY
  FROM (
    SELECT user_id,
      order_id,
      sale_price,
      created_at as order_created_date
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
  )
  GROUP BY user_id, order_id
 )
)

Finished job_id: b6970f49-8fa7-4edf-81b0-353ee4dc68e5


""


Let's take a look at the model's clustering performance, using these metrics - [Davies Bouldin Index](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index) and Mean Squared Distance

In [9]:
query = """
SELECT *
FROM ML.EVALUATE(MODEL `{0}.{1}`)
""".format(dataset_name, model_name)
run_bq_query(query)


Finished job_id: bb9e231b-b612-40ae-b4a1-2675acfeee53


,davies_bouldin_index,mean_squared_distance
0,0.901202,0.484058


Now let's get the cluster (centroid) information

In [10]:
query = """
SELECT
  CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
  avg_spend as average_spend,
  count_orders as count_of_orders,
  days_since_order
FROM (
  SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
  FROM ML.CENTROIDS(MODEL `{0}.{1}`)
)
PIVOT (
  SUM(value)
  FOR feature IN ('avg_spend',  'count_orders', 'days_since_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)
run_bq_query(query)

Finished job_id: 6024d3ba-07e2-4802-bd61-c64da26ea082


,centroid,average_spend,count_of_orders,days_since_order
0,cluster 1,152.08,1.0,810.29
1,cluster 2,426.64,1.0,824.58
2,cluster 3,48.90,1.0,1305.72
3,cluster 4,43.93,1.0,594.13
4,cluster 5,40.03,1.0,929.05


Whew! That's a lot of metrics and cluster info. How about we explain this to our colleagues using the magic of LLMs.

In [11]:
df = client.query(query).to_dataframe()
df.to_string(header=False, index=False)

cluster_info = []
for i, row in df.iterrows():
  cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
    .format(row["centroid"], row["count_of_orders"], row["average_spend"], row["days_since_order"]) )

print(str.join("\n", cluster_info))

cluster 1, average spend $152.08, count of orders per person 1.0, days since last order 810.29
cluster 2, average spend $426.64, count of orders per person 1.0, days since last order 824.58
cluster 3, average spend $48.9, count of orders per person 1.0, days since last order 1305.72
cluster 4, average spend $43.93, count of orders per person 1.0, days since last order 594.13
cluster 5, average spend $40.03, count of orders per person 1.0, days since last order 929.05


## Explain with Vertex AI Gemini API

Initialize the Vertex AI python SDK

In [12]:
import vertexai
vertexai.init(project=project_id, location=LOCATION)

Generate a text prediction

In [23]:
from vertexai.generative_models import GenerationConfig, GenerativeModel

model = GenerativeModel("gemini-1.0-pro")

clusters = str.join("\n", cluster_info)

prompt = f"""
You're a creative brand strategist, given the following clusters, come up with creative brand persona, a catchy title, and next marketing action, explained step by step.

Clusters:
{clusters}

For each Cluster:
* Title:
* Persona:
* Next Marketing Step:
"""

generation_config = GenerationConfig(
    temperature=0.55,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=1024,
)

responses = model.generate_content(
    prompt,
    generation_config=generation_config,
    #stream=True,
)
print(responses.candidates[0].content.parts[0].text)


**Cluster 1**

* **Title:** The Occasional Shopper
* **Persona:** Sarah is a busy professional who doesn't have much time for shopping. She typically makes a large purchase once a year, when she needs to replace something essential.
* **Next Marketing Step:** Reach out to Sarah with personalized emails offering exclusive discounts and promotions on items she's interested in.

**Cluster 2**

* **Title:** The Loyal Customer
* **Persona:** John is a retired homeowner who loves to shop. He's a regular customer who makes multiple purchases throughout the year.
* **Next Marketing Step:** Offer John a loyalty program that rewards him for his repeat business. Provide him with personalized recommendations and exclusive access to new products.

**Cluster 3**

* **Title:** The Budget-Conscious Shopper
* **Persona:** Mary is a single mother who is always looking for ways to save money. She typically only makes purchases when she finds a good deal.
* **Next Marketing Step:** Send Mary targeted emai

Voila!  🎉 We've now used k-means clustering to create groups of spenders and explain their profiles.

Sometimes, though, you want a little bit [extra](https://cloud.google.com/blog/transform/prompt-debunking-five-generative-ai-misconceptions).

In [24]:
from vertexai.generative_models import GenerationConfig, GenerativeModel

model = GenerativeModel("gemini-1.0-pro")

cluster_info = str.join('\n', cluster_info)

prompt = f"""
Pretend you're a creative strategist, analyse the following clusters and come up with \
creative brand persona for each that includes the detail of which Taylor Swift song is \
likely to be their favorite, a summary of how this relates to their purchasing behavior, \
and a witty e-mail headline for marketing campaign targeted to their group.

Clusters:
{cluster_info}
"""

#print(model.predict(
#    prompt,
#    max_output_tokens=1024,
#    temperature=0.45,
#    top_p=0.8, top_k=40,
#))
generation_config = GenerationConfig(
    temperature=0.55,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=1024,
)

responses = model.generate_content(
    prompt,
    generation_config=generation_config,
    #stream=True,
)
print(responses.candidates[0].content.parts[0].text)

**Cluster 1: The Occasional Treaters**

* **Brand Persona:**
    * Sarah, a busy professional in her mid-30s who values quality over quantity.
    * Enjoys the occasional indulgence but is mindful of her spending.
* **Favorite Taylor Swift Song:** "Style"
    * Relates to the song's sophisticated and elegant vibe.
* **Purchasing Behavior:**
    * Makes infrequent purchases, focusing on special occasions or when she needs to treat herself.
    * Prefers premium brands with a proven track record of quality.
* **Marketing Campaign Headline:** "Indulge in the Extraordinary, One Treat at a Time"

**Cluster 2: The Splurge Seekers**

* **Brand Persona:**
    * Emily, a fashion-forward millennial who loves to indulge in the finer things in life.
    * Has a discerning taste and is willing to pay top dollar for exclusive experiences.
* **Favorite Taylor Swift Song:** "Blank Space"
    * Enjoys the song's sassy and confident lyrics.
* **Purchasing Behavior:**
    * Makes frequent purchases, ofte

**Bonus inspiriation!**
If you'd like to use the PaLM or Gemini API directly in BigQuery, see this notebook [Using Vertex AI LLMs with data in BigQuery](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bigquery_ml_llm.ipynb) - and learn how to use the BigQuery function [`ML.GENERATE_TEXT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text). Using `ML_GENERATE_TEXT` you can combine cluster analysis and explainations all within BigQuery.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
